# Read input data

In [ ]:
import re
import pprint
import os
import numpy as np
from scipy.stats import chi2

# Define a pattern to match the relevant data
RE_IGNORE_OPTIONAL_LINE = r'(?:[^\n]*)?'
pattern = [
    r'Running ([/\w]+) on (\w+)[^\n]*',
    RE_IGNORE_OPTIONAL_LINE,
    RE_IGNORE_OPTIONAL_LINE,
    r'Number of threads: (\d+)',
    r'Runtime: (\d+\.[\d]+)'
]
pattern = r'\n?'.join(pattern) # This ? may cause unexpected behavior
print(f'Pattern: {pattern}')

# List all files in the 'data_runtime' directory
all_files = os.listdir('data_runtime')

# Filter files that start with 'runtime_'
runtime_files = [f for f in all_files if f.startswith('runtime_')]

runtimes = {}

for runtime_file in runtime_files:
    print(f'Parsing {runtime_file}...')
    with open(f'data_runtime/{runtime_file}', 'r') as file:
        file_format = file.read()

    # Find all matches in the string
    matches = re.findall(pattern, file_format)
    print(f'Found {len(matches)} matches')
    program = runtime_file[len('runtime_'):-4]
    
    # Convert matches to a list of dictionaries
    for match in matches:
        dataset = match[1]
        if runtimes.get(program) == None:
            runtimes[program] = {}
        if runtimes[program].get(dataset) == None:
            runtimes[program][dataset] = []
        runtimes[program][dataset].append({'binary': match[0], 'threads': int(match[2]), 'runtime': match[3]})

# pprint.pprint(runtimes)

for program, datasets in runtimes.items():
    for dataset, entries in datasets.items():
        runtime_values = [float(entry['runtime']) for entry in entries]
        # print(f'{program} on {dataset}: {runtime_values}')
        
        # Expected variance (threshold for acceptable noise)
        expected_variance = 0.1  # Adjust based on your requirements

        # Sample variance
        sample_variance = np.var(runtime_values, ddof=1)

        # Number of samples
        n = len(runtime_values)

        # Chi-square statistic
        chi2_stat = (n - 1) * sample_variance / expected_variance

        # p-value (two-tailed test)
        p_value = 2 * min(chi2.cdf(chi2_stat, n - 1), 1 - chi2.cdf(chi2_stat, n - 1))

        # print(f"Sample Variance: {sample_variance}")
        # print(f"Chi-Square Statistic: {chi2_stat}")
        # print(f"P-Value: {p_value}")

        # Interpretation
        alpha = 0.05  # Significance level
        if p_value < alpha:
            print("The variance is significantly different from the expected value.")
        else:
            print("The variance is NOT significantly different from the expected value.")


average_runtimes = {}
for program, datasets in runtimes.items():
    if program not in average_runtimes:
        average_runtimes[program] = {}
    for dataset, entries in datasets.items():
        total_runtime = sum(float(entry['runtime']) for entry in entries)
        average_runtime = total_runtime / len(entries)
        average_runtimes[program][dataset] = average_runtime

pprint.pprint(average_runtimes)

# Generate plots on runtime and speedups

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

dataset_category = {
    'Social_Network_1':         'small',
    'Web_Graph_1':              'small',
    'Collaboration_Network_1':  'small',
    'Synthetic_Dense_1':        'small',
    'Road_Network_1':           'large',
    'kNN_Graph_1':              'large',
    'Synthetic_Sparse_1':       'large',
    'Road_Network_2':           'verylarge',
}
category_colors = {
    'small': 'blue',
    'large': 'green',
    'verylarge': 'red',
}
datasets = list(dataset_category.keys())
programs = [
    'small96',
    'large96',
    'verylarge96',
    'auto96',

    # 'small24',
    # 'large24',
    # 'verylarge24',
    # 'auto24',
]

# Calculate speedups
speedups = {}
for program in programs:
    program_datasets = average_runtimes.get(program, {})
    speedups[program] = {}
    for dataset, runtime in program_datasets.items():
        if dataset in average_runtimes['reference']:
            speedups[program][dataset] = average_runtimes['reference'][dataset] / runtime

pprint.pprint(speedups)

# Prepare data for plotting
fig, ax = plt.subplots(figsize=(12, 10))
BAR_WIDTH = 0.1

# Plot speedups
for i, (program, program_speedups) in enumerate(speedups.items()):
    if program != 'reference':
        program_speedups = [program_speedups.get(dataset, np.nan) for dataset in datasets]
        bars = ax.bar(np.arange(len(datasets)) + i * BAR_WIDTH, program_speedups, BAR_WIDTH, label=program)
        for bar, speedup in zip(bars, program_speedups):
            if np.isnan(speedup):
                ax.text(bar.get_x() + bar.get_width() / 4, 0, 'x')
                

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('Datasets')
ax.set_ylabel('Speedup (Reference / Other)')
ax.set_title('Speedups by Dataset')
ax.set_yticks(np.arange(80))
ax.set_yticklabels([f'{i}x' for i in range(80)])
ax.set_xticks(np.arange(len(datasets)) + BAR_WIDTH * (len(speedups) - 1) / 2)
ax.set_xticklabels(datasets, rotation=45, ha='right')
for tick, dataset in zip(ax.get_xticklabels(), datasets):
    tick.set_color(category_colors[dataset_category[dataset]])
ax.legend(loc='best')
ax.grid()


fig.tight_layout()
plt.show()